In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
print('gerando datas')
datas = pd.date_range(start = '2011-01-01', end = pd.to_datetime('today'), freq = 'M')

d = {}
str_datas = [str(x) for x in datas]

for dt in str_datas:    
    header = {'ACCEPT':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
    baseURL = f'https://www.npr.org/series/423302056/hidden-brain/archive?date={dt}'
    req = requests.request('GET', baseURL, headers = header, data = {})

    soup = BeautifulSoup(req.text)

    for x in soup.find_all():
            if f'npr.org/{dt[:4]}/' in str(x.get('href')) and len(x.get_text()) > 3 :
                link = x.get('href')
                text = x.get_text()
                if link not in list(d):
                    d[link] = [text]    
                else:
                    if text not in d[link]:
                        d[link].append(x.get_text())

    

gerando datas


In [7]:
pd.DataFrame.from_dict(d, orient = 'index')

,0,1,2
https://www.npr.org/2011/06/10/137112887/some-suggest-power-increases-promiscuity,\n\n\n\n\n\n,Power May Increase Promiscuity,"June 10, 2011  Anthony Weiner, Arnold Schwarz..."
https://www.npr.org/2011/07/07/137660609/its-a-duel-how-do-violent-video-games-affect-kids,\n\n\n\n\n\n,Dispute: How Violent Video Games Affect Kids,"July 7, 2011  Scientists have long clashed ov..."
https://www.npr.org/2011/07/04/137526401/the-key-to-disaster-survival-friends-and-neighbors,\n\n\n\n\n\n,"The Key To Disaster Survival? Pals, Neighbors","July 4, 2011  A researcher's data suggest tha..."
https://www.npr.org/2011/08/31/140085048/to-dodge-blame-officials-prepare-public-for-worst,\n\n\n\n\n\n,"To Dodge Blame, Officials Prep Public For Worst","August 31, 2011  Because people are more like..."
https://www.npr.org/2011/08/02/138922339/under-pressure-soccer-goalies-tend-to-dive-right,\n\n\n\n\n\n,"Under Pressure, Soccer Goalies Tend To Dive Right","August 2, 2011  Here's some science for the b..."
...,...,...,...
https://www.npr.org/2020/07/31/897673162/you-2-0-our-pursuit-of-happiness,\n\n\n\n\n\n,You 2.0: Our Pursuit of Happiness,"August 3, 2020  Sometimes, life can feel like..."
https://www.npr.org/2020/09/25/916997530/laughter-the-best-medicine,\n\n\n\n\n\n,Laughter: The Best Medicine,"September 28, 2020  If you listen closely to ..."
https://www.npr.org/2020/08/21/904660038/the-halo-effect-why-its-so-difficult-to-understand-the-past,\n\n\n\n\n\n,The Halo Effect,"September 21, 2020  Judy, Lyn and Donna Ulric..."
https://www.npr.org/2020/09/14/912749547/why-nobody-feels-rich-the-psychology-of-inequality,\n\n\n\n\n\n,Why Nobody Feels Rich,"September 14, 2020  If you've ever flown in e..."


In [127]:
import pandas as pd 
import re 
df = pd.DataFrame.from_dict(d, orient = 'index')
df = df.drop(columns = [0])
df = df.reset_index()
df.columns  = ['link','title','description']
# df['date'] = pd.to_datetime(df.description.apply(lambda x: re.findall('[A-Za-z0-9\s,]*[\d]{4}',x)[0]))



# df['description'] = df.description.apply(lambda x: re.findall('\d{4}..(.*)',x)[0])
# df = df.sort_values('date', ascending = False)


df['date'] = df['description'].str.extract('(\w{3,14} \d{1,2}, \d{4})')
df['date'].replace('^(\w+) (\d+), (\d+).*$', r'\3 \2 \1', regex = True)
df['date'] = pd.to_datetime(df['date'].replace('^(\w+) (\d+), (\d+).*$', r' \3 \1 \2', regex = True))
df['description'] = df.description.replace('^\w+ \d+, \d+ . (.*)$',r'\1', regex = True)
df


,link,title,description,date
0,https://www.npr.org/2011/06/10/137112887/some-...,Power May Increase Promiscuity,"Anthony Weiner, Arnold Schwarzenegger — men be...",2011-06-10
1,https://www.npr.org/2011/07/07/137660609/its-a...,Dispute: How Violent Video Games Affect Kids,Scientists have long clashed over whether viol...,2011-07-07
2,https://www.npr.org/2011/07/04/137526401/the-k...,"The Key To Disaster Survival? Pals, Neighbors","A researcher's data suggest that ambulances, f...",2011-07-04
3,https://www.npr.org/2011/08/31/140085048/to-do...,"To Dodge Blame, Officials Prep Public For Worst",Because people are more likely to blame public...,2011-08-31
4,https://www.npr.org/2011/08/02/138922339/under...,"Under Pressure, Soccer Goalies Tend To Dive Right",Here's some science for the betting table: Whe...,2011-08-02
...,...,...,...,...
603,https://www.npr.org/2020/07/31/897673162/you-2...,You 2.0: Our Pursuit of Happiness,"Sometimes, life can feel like being stuck on a...",2020-08-03
604,https://www.npr.org/2020/09/25/916997530/laugh...,Laughter: The Best Medicine,"If you listen closely to giggles, guffaws, and...",2020-09-28
605,https://www.npr.org/2020/08/21/904660038/the-h...,The Halo Effect,"Judy, Lyn and Donna Ulrich were driving to a v...",2020-09-21
606,https://www.npr.org/2020/09/14/912749547/why-n...,Why Nobody Feels Rich,If you've ever flown in economy class on a pla...,2020-09-14
